In [19]:
import tensorflow as tf
from tensorflow.keras import datasets
import numpy as np
import cv2
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Load CIFAR-10 dataset

In [20]:
def preprocess(images):
    images = images.astype('float32') / 255.0
    return images

In [21]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []
    for image in images:
        image = (image * 255).astype(np.uint8)  # Convert image to uint8
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None and len(keypoints) > 0:  # Ensure valid descriptors
            descriptors_list.append(descriptors)
            keypoints_list.append(keypoints)
    return keypoints_list, descriptors_list

In [22]:
def bow_representation(keypoints, descriptors, kmeans):
    bow_representation = np.zeros((len(keypoints), kmeans.n_clusters), dtype=np.float32)
    for i in range(len(keypoints)):
        if descriptors[i] is not None:
            words = kmeans.predict(descriptors[i])
            for word in words:
                bow_representation[i, word] += 1
    return bow_representation

In [23]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Preprocessing

train_images = preprocess(train_images)
test_images = preprocess(test_images)

# Feature extraction using SIFT


train_keypoints, train_descriptors = extract_sift_features(train_images)
test_keypoints, test_descriptors = extract_sift_features(test_images)

# Flatten descriptors
train_descriptors_flat = np.vstack(train_descriptors)
test_descriptors_flat = np.vstack(test_descriptors)


In [26]:
kmeans = KMeans(n_clusters=512)
kmeans.fit(train_descriptors_flat)

# Generate Bag-of-Visual Words representation


train_bow = bow_representation(train_keypoints, train_descriptors, kmeans)
test_bow = bow_representation(test_keypoints, test_descriptors, kmeans)

# Scale features
scaler = StandardScaler()
train_bow_scaled = scaler.fit_transform(train_bow)
test_bow_scaled = scaler.transform(test_bow)

In [17]:
import optuna

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-10, 1e10)
    # kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    svm = SVC(C=C, kernel='linear')
    
    # Fit the classifier to the training data
    svm.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

    # Make predictions on the training and test data
    train_predictions = svm.predict(train_bow_scaled)
    test_predictions = svm.predict(test_bow_scaled)

    # Calculate accuracy
    train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
    test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

    # Return the test accuracy as the objective value to maximize
    return test_accuracy

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
best_svm = SVC(**best_params)

# Fit the classifier with the best hyperparameters
best_svm.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Make predictions on the test data using the best classifier
test_predictions = best_svm.predict(test_bow_scaled)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

print("Best hyperparameters:", best_params)
print("Test Accuracy with best model:", test_accuracy)


[I 2024-02-15 11:19:29,556] A new study created in memory with name: no-name-6974057b-262e-472d-9afd-1649a8af57f6
C:\Users\Srinivasan M\AppData\Local\Temp\ipykernel_13888\987338433.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
[I 2024-02-15 11:26:19,936] Trial 0 finished with value: 0.09983961507618284 and parameters: {'C': 3.4405200857798846e-10}. Best is trial 0 with value: 0.09983961507618284.
C:\Users\Srinivasan M\AppData\Local\Temp\ipykernel_13888\987338433.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
[I 2024-02-15 11:33:21,533] Trial 1 finished with value: 0.

KeyboardInterrupt: 

In [27]:
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-10, 1e10)
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    
    # Initialize the classifier with the suggested hyperparameters
    clf = LogisticRegression(C=C, max_iter=max_iter, random_state=42)
    
    # Fit the classifier to the training data
    clf.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

    # Make predictions on the training and test data
    train_predictions = clf.predict(train_bow_scaled)
    test_predictions = clf.predict(test_bow_scaled)

    # Calculate accuracy
    train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
    test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

    # Return the test accuracy as the objective value to maximize
    return test_accuracy

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
best_lr = LogisticRegression(**best_params, random_state=42)

# Fit the classifier with the best hyperparameters
best_lr.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Make predictions on the test data using the best classifier
test_predictions = best_lr.predict(test_bow_scaled)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

print("Best hyperparameters:", best_params)
print("Test Accuracy with best model:", test_accuracy)


[I 2024-02-15 11:41:20,048] A new study created in memory with name: no-name-82265fbf-dddb-4f39-8531-314af29237a8
C:\Users\Srinivasan M\AppData\Local\Temp\ipykernel_13888\2190131342.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
[I 2024-02-15 11:41:20,313] Trial 0 finished with value: 0.09873696872493985 and parameters: {'C': 0.0729990374975001, 'max_iter': 244}. Best is trial 0 with value: 0.09873696872493985.
C:\Users\Srinivasan M\AppData\Local\Temp\ipykernel_13888\2190131342.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-10, 1e10)
[I 2024-02-15 11:41:20,431] Trial 1 finished

Best hyperparameters: {'C': 8.620385894156097e-05, 'max_iter': 522}
Test Accuracy with best model: 0.10184442662389735


In [28]:
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 100)
    knn_clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    
    # Fit the classifier to the training data
    knn_clf.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

    # Make predictions on the training and test data
    train_predictions = knn_clf.predict(train_bow_scaled)
    test_predictions = knn_clf.predict(test_bow_scaled)

    # Calculate accuracy
    train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
    test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

    # Return the test accuracy as the objective value to maximize
    return test_accuracy

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
best_knn = KNeighborsClassifier(**best_params)

# Fit the classifier with the best hyperparameters
best_knn.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Make predictions on the test data using the best classifier
test_predictions = best_knn.predict(test_bow_scaled)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

print("Best hyperparameters:", best_params)
print("Test Accuracy with best model:", test_accuracy)

[I 2024-02-15 11:41:59,117] A new study created in memory with name: no-name-46666e36-4421-435a-84e9-19ba87746703
[I 2024-02-15 11:42:05,196] Trial 0 finished with value: 0.10154370489174018 and parameters: {'n_neighbors': 6}. Best is trial 0 with value: 0.10154370489174018.
[I 2024-02-15 11:42:17,136] Trial 1 finished with value: 0.10274659182036888 and parameters: {'n_neighbors': 77}. Best is trial 1 with value: 0.10274659182036888.
[I 2024-02-15 11:42:26,872] Trial 2 finished with value: 0.1047514033680834 and parameters: {'n_neighbors': 59}. Best is trial 2 with value: 0.1047514033680834.
[I 2024-02-15 11:42:32,475] Trial 3 finished with value: 0.10154370489174018 and parameters: {'n_neighbors': 6}. Best is trial 2 with value: 0.1047514033680834.
[I 2024-02-15 11:42:45,621] Trial 4 finished with value: 0.1008420208500401 and parameters: {'n_neighbors': 86}. Best is trial 2 with value: 0.1047514033680834.
[I 2024-02-15 11:43:02,276] Trial 5 finished with value: 0.10224538893344026 a

Best hyperparameters: {'n_neighbors': 37}
Test Accuracy with best model: 0.10926222935044105
